# Team Members
Team Name - Team Seven
1. Reezwan Ul Haq Mohammad - rmoham27@uic.edu
2. Sai Anish Garapati - sgarap2@uic.edu
3. Vamsi Krishna Mandalapu - vmanda8@uic.edu

Project Name - Crash Prediction in Chicago

# Any Changes
Previously we wanted to build models around the usecase of predicting the fatality of the accident, but the number of accidents that involve serious fatalities are less. So we are building models around the usecase of predicting the 'Primary Reason for a Crash'

# Data
We are using 3 different datasets which are crashes dataset, vehicle dataset and people dataset. These datasets have the details of a crash occurred, vehicles involved in the crash and people invloved in the crash.
There are 700k rows with 49 columns of data in Crash Dataset.
Original size of vehicle dataset is 780 MB which has 1.5 million rows with 71 columns.
The people dataset contains information about people involved in a crash and if any injuries were sustained.
We are using the crash id which is contained in all the datasets to map them where if two or more vehicles are included in a crash, we pick the vehicle which is the reason for the crash.
Source

Crashes Dataset - https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if

Vehicle Dataset - https://data.cityofchicago.org/Transportation/Traffic-Crashes-Vehicles/68nd-jvt3

People Dataset - https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d
